In [ ]:
import requests
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from datetime import datetime

This API provides access to the JPL/SSD small-body mission design suite. The following operation modes are available:

* Mode A (accessible) - retrieves the list of accessible small bodies based on user-defined constraint.
* Mode Q (query) - retrieves pre-computed mission options to a specific object. Both impulsive and low-thrust gravity-assist mission options are available.
* Mode M (map) - an extension of mode Q for the impulsive case, returns the data required to render a porkchop plot with multiple parameters.
* Mode T (mission extension) - retrieves the list of small bodies that come closest (or within a prescribed distance) to a user-defined orbit during a certain period of time. This is a crude filter for finding potential candidates for mission extensions.

This script emphazise the development of the following mode

* Mode M - In addition to querying the database like in mode Q (ballistic), compute all ballistic mission options to the specified target within certain ranges of launch dates and times of flight.
    * In addition, the values of the x-_y_ axes of the maps are also provided:
        * dep_date - departure dates from Earth (Modified Julian Date), corresponding to the x-axis.
        * tof - times of flight to the target (days), corresponding to the y-axis.
        * If dep_date has m elements and tof has n, then the 2D arrays are of dimension n x m.
        * vinf_dep
        * vinf_arr

# Single example Query 

In [ ]:
url_base = 'https://ssd-api.jpl.nasa.gov/mdesign.api'
asteroid_name = '2012TC4' # designation (provisional or IAU-number) of the desired object (e.g., 2015 AB or 141P or 433).
                         # NOTE: when submitting a des containing a space in your query string, you must 
                         # replace the space with %20, for example 2015%20AB.
mjd0 = 59215            # 21Jan2021 # first launch date to be explored (Modified Julian Date)
span = 365                # duration of the launch-date period to be explored (days)
tof_min = 50          # minimum time of flight to be considered (days)
tof_max = 720       # maximum time of flight to be considered (days)
step = 5            # time step used to advance both the launch date and the time of flight (days). 
                    # The size of the transfer map is limited to 1,500,000 points

sim_lim_points = 1500000 #1.5 millions
if int(span)/int(step) > sim_lim_points:
    print('outside of tool limits') 

url = f'{url_base}?sstr={str(asteroid_name)}&mjd0={str(mjd0)}&span={str(span)}&tof-min={str(tof_min)}&tof-max={str(tof_max)}&step={str(step)}'
r = requests.get(url)

data = r.json()

print(data)

In [ ]:
def get_mission_profiles(asteroid_name,mjd0,span,tof_min,tof_max,step):
    # asteroid_name:    designation (provisional or IAU-number) of the desired object (e.g., 2015 AB or 141P or 433).
    #                   NOTE: when submitting a des containing a space in your query string, you must replace the space with %20, for example 2015%20AB
    # mjd0:             first launch date to be explored (Modified Julian Date)
    # span:             duration of the launch-date period to be explored (days)
    # tof-min:          minimum time of flight to be considered (days)
    # tof-max:          maximum time of flight to be considered (days)
    # step:             time step used to advance both the launch date and the time of flight (days). 
                        
    # The size of the transfer map is limited to 1,500,000 points
    sim_lim_points = 1500000 #1.5 millions
    if int(span)/int(step) > sim_lim_points:
        print('outside of tool limits') # TODO return error
    
    # Construction of the HTTP request
    url_base = 'https://ssd-api.jpl.nasa.gov/mdesign.api'
    url = f'{url_base}?sstr={str(asteroid_name)}&mjd0={str(mjd0)}&span={str(span)}&tof-min={str(tof_min)}&tof-max={str(tof_max)}&step={str(step)}'
    r = requests.get(url)
    data = r.json()
    
    # Elaboration of data
    available_missions = len(data['selectedMissions'])
    mission_profiles={};
    for mission_id in range(available_missions):
        if (data["selectedMissions"][mission_id][0] > 59215 and data["selectedMissions"][mission_id][1] < 69076):
            sel_profile={"fullname": data["object"]["fullname"],
                      "mjd0": data["selectedMissions"][mission_id][0],
                      "mjdf": data["selectedMissions"][mission_id][1],
                      "tof": data["selectedMissions"][mission_id][9],
                      "vinf_dep": data["selectedMissions"][mission_id][2],
                      "vinf_arr": data["selectedMissions"][mission_id][3],
                      "earth_dist": data["selectedMissions"][mission_id][5],
                      "phase_ang": data["selectedMissions"][mission_id][4], 
                      "elong_arr": data["selectedMissions"][mission_id][6], 
                      "decl_dep": data["selectedMissions"][mission_id][7],
                      "approach_ang": data["selectedMissions"][mission_id][8],      
                     };
        mission_profiles[mission_id]=sel_profile;
    
    # porkchop
    porkchop_dv, dep_date, tof = get_mission_porkchop(data);
    #plt.contour(dep_date, tof, porckchop_dv)
    return mission_profiles, porkchop_dv, dep_date, tof

In [ ]:
def get_mission_porkchop(data):
    dep_date=data["dep_date"];
    tof=data["tof"];
    # Elaboration of data
    m = len(data["dep_date"])
    n = len(data["tof"])
    #porkchop_map=[];
    porkchop_map = np.zeros([n,m]) # porkchop_map[i,j]
    for i in range(n):
        for j in range(m):
            porkchop_map[i,j]=abs(data["vinf_arr"][i][j])+abs(data["vinf_dep"][i][j])
    return porkchop_map, dep_date, tof

In [ ]:
missions_2012tc4, porkchop_dv_2012tc4, dep_date_2012tc4, tof_2012tc4 = get_mission_profiles('2012TC4',59215,365,50,700,5)

In [ ]:
# minimum of the selectedMissions profiles
dv = np.zeros(len(missions_2012tc4))
for profile in missions_2012tc4:
    dv[profile] = missions_2012tc4[profile]['vinf_dep'] + missions_2012tc4[profile]['vinf_arr']

index = np.linspace(0,len(dv)-1,len(dv))
mask = [dv == np.min(dv)]
index_min = index[mask]
missions_2012tc4[index_min[0]]

In [ ]:
fig = plt.figure()
plt.plot(dv, "*");
fig.suptitle('Mission Profile dv Distribution for '+ missions_2012tc4[0]["fullname"])
plt.xlabel('$idx$ (-)')
plt.ylabel('$dv$ (km/s)')
plt.plot(int(index_min[0]), dv[int(index_min[0])], "r+");

In [ ]:
fig = plt.figure()
plt.contour(dep_date_2012tc4, tof_2012tc4, porkchop_dv_2012tc4, np.linspace(0,50,51),cmap="gnuplot")
fig.suptitle('Porkchop Plot for '+ missions_2012tc4[0]["fullname"])
plt.xlabel('$Date_{dep}$ (mjd)')
plt.ylabel('$ToF$ (d)')                               
plt.colorbar();